Urban Data Science & Smart Cities <br>
URSP688Y <br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

[<img src="https://colab.research.google.com/assets/colab-badge.svg">](https://colab.research.google.com/github/ncsg/ursp688y_sp2024/blob/main/exercises/exercise06/exercise06.ipynb)

# Exercise 6

## Problem

In class this week, we saw how to access real-time data about Capital Bikeshare from the internet using their API. We also dealt with the challenge of wrangling those data. We needed to parse a JSON file into a table, and we considered how we might retrieve, store, and combine many JSONs in order to understand how bike availability changed over time.

These real-time data can help us answer questions about how well Captial Bikeshare is being utilized.

See if you can use data from the API (I have already stored and combined it--see below) answer these questions:
- How many bikes were available within the system during each hour over a 24 hour period?
    - Can you graph this over time?
    - Which hour of the day were bikes most available? Least available?

**Bonus:** Can you write a function to estimate how many bikes are <ins>currently being used</ins>, whenever you call the function? This will require loading real-time data from the API and comparing it to stored data.

## Data

I wrote a script, which you can see [here](https://github.com/ncsg/ursp688y_sp2024/blob/main/demos/demo06/cabi_data/get_cabi_free_bikes.py), to retrieve and store JSON data from the `free_bike_status` table in [Capital Bikeshare's](https://capitalbikeshare.com/system-data) GBFS feed every 5 minutes. I ran this script on my computer for a bit more than 24 hours. ([Here's a tutorial](https://realpython.com/run-python-scripts/) on running scripts on the command line, if you're curious.) All of those JSONS are available for you to use. They're stored at [`ursp688y_sp2024/demos/demo06/cabi_data`](https://github.com/ncsg/ursp688y_sp2024/tree/main/demos/demo06/cabi_data).

## Building Off of the Demo

The in-class demo gave us a starting point for how to access real-time JSON data from the API, load saved JSON data, and parse JSON data into a DataFrame.

I have copied what we did in class below and added onto it to develop a single tidy dataframe with records from all the saved JSONs, plus timestamps. This should be all the data you need for the questions above (except the bonus).

See if you can follow my code, then build onto it.

As usual, please wrap the code for your solution in a function, and put that function into a module (you can add to my module, or make a new one if you prefer). Then load your main function from the module and call it in the notebook to demonstrate your solution.


# Setup

In [1]:
# Import packages
import os
import json
import requests
import pandas as pd

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Set the working directory
# You will need to change this to your own folder on Google Drive
os.chdir('/content/drive/MyDrive/Colab Notebooks/Exercise6_Bardsley')

In [3]:
# Import module
import exercise06

# Request current data from the API

In [4]:
# Making a get request
response = requests.get('https://gbfs.lyft.com/gbfs/1.1/dca-cabi/en/free_bike_status.json')

# Get JSON content
data = response.json()

# Inspect the contents
data.keys()

dict_keys(['data', 'last_updated', 'ttl', 'version'])

In [7]:
data['data'].keys()

dict_keys(['bikes'])

In [5]:
# Make a dataframe out of data for available bikes
df = pd.DataFrame(data['data']['bikes'])

df.head()

,name,rental_uris,is_reserved,is_disabled,bike_id,lon,fusion_lon,type,fusion_lat,lat
0,222-581,{'android': 'https://dc.lft.to/lastmile_qr_sca...,0,0,093ee410f358e7576732f5d14a4c676c,-76.998711,0.0,electric_bike,0.0,38.919061
1,201-455,{'android': 'https://dc.lft.to/lastmile_qr_sca...,0,0,1dfbdf8fa93a5da4efe9ec3aa629aff9,-77.147755,0.0,electric_bike,0.0,38.863876
2,623-376,{'android': 'https://dc.lft.to/lastmile_qr_sca...,0,0,c87860d005443bcf0a8c3862ff4a133d,-77.010481,0.0,electric_bike,0.0,38.877649
3,170-725,{'android': 'https://dc.lft.to/lastmile_qr_sca...,0,0,2c07c43f06d2ce6370ec292145007de1,-77.025999,0.0,electric_bike,0.0,38.899957
4,122-633,{'android': 'https://dc.lft.to/lastmile_qr_sca...,0,0,dc2df69f0b72e65eb3106738024683ca,-77.027473,0.0,electric_bike,0.0,38.935759


# Load JSON data saved in a file

In [24]:
f = requests.get('https://raw.githubusercontent.com/ncsg/ursp688y_sp2024/main/demos/demo06/cabi_data/cabi_bike_status_2024-03-03_13-11-54.json')
data = f.json()


In [25]:
# see how the data are stored
type(data)

dict

In [26]:
# see what keys are available
data.keys()

dict_keys(['data', 'last_updated', 'ttl', 'version'])

In [27]:
# drill into the records for each bike
records = data['data']['bikes']

# convert to a dataframe
df = pd.DataFrame(records)

# drop a column that we won't use, just to keep things clean
df = df.drop(columns=['rental_uris'])

In [28]:
df.head()

,is_reserved,fusion_lon,fusion_lat,lat,type,is_disabled,bike_id,name,lon
0,0,0.0,0.0,38.887458,electric_bike,0,d94788433d337e4186fb431076b52e91,320-065,-77.025747
1,0,0.0,0.0,38.905328,electric_bike,0,cc49246f85fdc23a6a13b3402ab52b37,222-581,-77.058526
2,0,0.0,0.0,38.908954,electric_bike,0,75c5df17a8236707a7948f509a5ab929,228-812,-77.043055
3,0,0.0,0.0,38.955421,electric_bike,0,4e51a79c1e03962064762ff16013b1a8,570-760,-76.940135
4,0,0.0,0.0,38.892292,electric_bike,0,5983a1b66f086f7905d8aa701fa7b5df,268-224,-77.042912


# Iteratively load all the JSON files and combine them into a single dataframe

Except for the impact statements above, this is probably the only part of the code you'll need to keep. This function wraps all the loading steps. Feel free to delete the cells above if you're not using them.

In [4]:
df = exercise06.load_and_combine_free_bike_status_jsons_as_df('cabi_data')

df.head()

,is_reserved,fusion_lon,fusion_lat,lat,type,is_disabled,bike_id,name,lon,timestamp
0,0,0.0,0.0,38.887460,electric_bike,0,9fbc66c297f988ec843bdff54b7243fe,320-065,-77.025774,2024-03-03 18:21:20-05:00
1,0,0.0,0.0,38.905291,electric_bike,0,94ecae73dacebfeb5cb6dd641c77f233,222-581,-77.058555,2024-03-03 18:21:20-05:00
2,0,0.0,0.0,38.955477,electric_bike,0,39f687453d9505f86bfe80c20774eae1,570-760,-76.940137,2024-03-03 18:21:20-05:00
3,0,0.0,0.0,38.907830,electric_bike,0,c3e9c0b941ed726e768e9703c0ff7fb3,137-726,-77.071648,2024-03-03 18:21:20-05:00
4,0,0.0,0.0,38.863824,electric_bike,0,a3f4b95048b95c438471b3c078ccbb86,201-455,-77.147790,2024-03-03 18:21:20-05:00


This is where you take over. Can you use this dataframe to answer the question(s) above?

In [21]:
#Find the minimum hour
df['timestamp'].min()
#Find the maximum hour
df['timestamp'].max()
#convert date, which is a series, to a string
from datetime import datetime as dt
df['DateString'] = df['timestamp'].astype('string')
#get rid of the extra stuff in the timestamp that stops
#it from converting to datetime format
df['DateString2'] = df['DateString'].str.slice(start=0, stop=19)
#convert to datetime format
df['DateTime'] = pd.to_datetime(df['DateString2'])
#convert to ordinal time
df['OrdinalTime'] = df['DateTime'].apply(lambda x: x.toordinal())
df.head()

#Next steps:
#1. Create function that counts the number of unique values in column
#df['name'] when df['OrdinalTime'] is within a range that indicates each
#hour in the 24 hour period. Perhaps use a mask for each hour?
#Output this as a new dictionary with key = hour,
#value = count.
#2. Convert dictionary into pandas dataframe
#3. Use Seaborn to create bar plot from the dataframe

,is_reserved,fusion_lon,fusion_lat,lat,type,is_disabled,bike_id,name,lon,timestamp,DateString,DateString2,DateTime,OrdinalTime
0,0,0.0,0.0,38.887460,electric_bike,0,9fbc66c297f988ec843bdff54b7243fe,320-065,-77.025774,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20,2024-03-03 18:21:20,738948
1,0,0.0,0.0,38.905291,electric_bike,0,94ecae73dacebfeb5cb6dd641c77f233,222-581,-77.058555,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20,2024-03-03 18:21:20,738948
2,0,0.0,0.0,38.955477,electric_bike,0,39f687453d9505f86bfe80c20774eae1,570-760,-76.940137,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20,2024-03-03 18:21:20,738948
3,0,0.0,0.0,38.907830,electric_bike,0,c3e9c0b941ed726e768e9703c0ff7fb3,137-726,-77.071648,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20,2024-03-03 18:21:20,738948
4,0,0.0,0.0,38.863824,electric_bike,0,a3f4b95048b95c438471b3c078ccbb86,201-455,-77.147790,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20-05:00,2024-03-03 18:21:20,2024-03-03 18:21:20,738948
